**Lab 01 (AI-501 MFAI) RollNo_25201317**

**Linear discriminant analysis (LDA)**

In [29]:
import numpy as np
import pandas as pd
np.set_printoptions(precision=3, suppress=True)

data = {
    "Infection": ["Viral","Viral","Viral","Viral","Viral","Viral",
                  "Bacterial","Bacterial","Bacterial","Bacterial","Bacterial","Bacterial"],
    "CRP": [40.0, 11.1, 30.0, 21.4, 10.7, 3.4, 42.0, 31.1, 50.0, 60.4, 45.7, 17.3],
    "Temp":[36.0, 37.2, 36.5, 39.4, 39.6, 40.7, 37.6, 42.2, 38.5, 39.4, 38.6, 42.7]
}
df = pd.DataFrame(data)
print("\nInput Data:\n", df, "\n")


Input Data:
     Infection   CRP  Temp
0       Viral  40.0  36.0
1       Viral  11.1  37.2
2       Viral  30.0  36.5
3       Viral  21.4  39.4
4       Viral  10.7  39.6
5       Viral   3.4  40.7
6   Bacterial  42.0  37.6
7   Bacterial  31.1  42.2
8   Bacterial  50.0  38.5
9   Bacterial  60.4  39.4
10  Bacterial  45.7  38.6
11  Bacterial  17.3  42.7 



In [30]:
viral = df[df["Infection"]=="Viral"][["CRP","Temp"]].to_numpy()
bact  = df[df["Infection"]=="Bacterial"][["CRP","Temp"]].to_numpy()
n1, n2 = viral.shape[0], bact.shape[0]

In [31]:
mean_viral = viral.mean(axis=0)
mean_bact  = bact.mean(axis=0)
overall_mean = df[["CRP","Temp"]].mean().to_numpy()
print("Mean (Viral):", np.round(mean_viral, 3))
print("Mean (Bacterial):", np.round(mean_bact, 3))
print("Overall Mean:", np.round(overall_mean, 3), "\n")

Mean (Viral): [19.433 38.233]
Mean (Bacterial): [41.083 39.833]
Overall Mean: [30.258 39.033] 



In [32]:
cov_viral = np.cov(viral.T, bias=True)
cov_bact  = np.cov(bact.T, bias=True)
print("Covariance (Viral):\n", np.round(cov_viral, 3))
print("Covariance (Bacterial):\n", np.round(cov_bact, 3), "\n")

Sw = (n1 * cov_viral + n2 * cov_bact) / (n1 + n2)
print("Within-Group Covariance Matrix S_w:\n", np.round(Sw, 3), "\n")

mean_diff = (mean_viral - mean_bact).reshape(-1, 1)
Sb = mean_diff @ mean_diff.T
print("Between-Group Covariance Matrix S_b:\n", np.round(Sb, 3), "\n")

Covariance (Viral):
 [[156.882 -17.471]
 [-17.471   3.062]]
Covariance (Bacterial):
 [[190.018 -19.968]
 [-19.968   3.716]] 

Within-Group Covariance Matrix S_w:
 [[173.45  -18.719]
 [-18.719   3.389]] 

Between-Group Covariance Matrix S_b:
 [[468.723  34.64 ]
 [ 34.64    2.56 ]] 



In [33]:
eigvals, eigvecs = np.linalg.eig(np.linalg.inv(Sw).dot(Sb))
idx = np.argmax(eigvals.real)
w_raw = eigvecs[:, idx].real
w_raw = w_raw / np.linalg.norm(w_raw)
print("Eigenvalues:", np.round(eigvals.real, 3))
print("Eigenvector (Discriminant Coefficients):", np.round(w_raw, 4), "\n")

print("Discriminant Function (Raw): D = {:.4f}*CRP + {:.4f}*Temp".format(w_raw[0], w_raw[1]))

Eigenvalues: [14.025 -0.   ]
Eigenvector (Discriminant Coefficients): [0.15  0.989] 

Discriminant Function (Raw): D = 0.1496*CRP + 0.9887*Temp


In [34]:
stds = df[["CRP","Temp"]].std(ddof=0).to_numpy()
Z = (df[["CRP","Temp"]].to_numpy() - overall_mean) / stds

cov_viral_z = np.cov(Z[:n1].T, bias=True)
cov_bact_z  = np.cov(Z[n1:].T, bias=True)
Sw_z = (n1 * cov_viral_z + n2 * cov_bact_z) / (n1 + n2)
mean_diff_z = (Z[:n1].mean(axis=0) - Z[n1:].mean(axis=0)).reshape(-1, 1)
Sb_z = mean_diff_z @ mean_diff_z.T

eigvals_z, eigvecs_z = np.linalg.eig(np.linalg.inv(Sw_z).dot(Sb_z))
idx_z = np.argmax(eigvals_z.real)
w_z = eigvecs_z[:, idx_z].real
if w_z[0] < 0: w_z = -w_z
w_z_unit = w_z / np.linalg.norm(w_z)
print("\nStandardized Within-Group Covariance S_w_z:\n", np.round(Sw_z, 3))
print("Standardized Between-Group Covariance S_b_z:\n", np.round(Sb_z, 3))
print("Eigenvalues (Standardized):", np.round(eigvals_z.real, 3))
print("Discriminant Coefficients (Z, unit length):", np.round(w_z_unit, 4))


Standardized Within-Group Covariance S_w_z:
 [[ 0.597 -0.547]
 [-0.547  0.841]]
Standardized Between-Group Covariance S_b_z:
 [[1.613 1.012]
 [1.012 0.635]]
Eigenvalues (Standardized): [14.025 -0.   ]
Discriminant Coefficients (Z, unit length): [0.789 0.614]


In [35]:
mean_score_viral = np.dot(mean_viral, w_raw)
mean_score_bact  = np.dot(mean_bact, w_raw)
cutoff = (mean_score_viral + mean_score_bact) / 2
print("\nCutoff Score (Raw): {:.3f}".format(cutoff))


Cutoff Score (Raw): 43.121
